# Create our Master Dataframe

In [1]:
# imports
import requests
import numpy as np
import seaborn as sb
import pandas as pd
from bs4 import BeautifulSoup
import re
import urllib
import time
from io import StringIO

### Use Beautiful Soup to get Intial Dataframe

In [3]:
def getMovieData(listOfURLS):

    # create list for our dataframes
    dfList = []

    # iterate through the urls
    for url in listOfURLS:
        # Extract website html using beatiful soup
        boxOffice2024 = requests.get(url).text
        movieSoup = BeautifulSoup(boxOffice2024, "html")

        # read in dataframe using read_html and beatuful soup to extract the table from the website. Wrap it in stringIO to get rid of
        # warning signs
        df = pd.read_html(StringIO(str(movieSoup.find_all("table"))))
        # The results come in the form of a list with a single item so we need to extract the table 
        df = df[0]
        # Drop the columns we don't need
        df = df.drop(columns = ["Genre","Budget","Running Time","Theaters","Total Gross","Estimated","Rank"])
        # Rename the columns
        df.columns = ["Movie Title", "Gross Earnings ($)", "Release Date", "Distributor"]
        # Create a new column with the year the movie was made using a regex to get the year from the url and making it an integer
        df["Year"] = int(re.findall(r"\d{4}",url)[0])
        #chatGPT request("How to convert dollar signs to integer")
        # We want to convert the gross earnings in the $ format to integers so we can do analysis on them easier
        # I knew how to do the .astype method but i wanted an easier way to do the regex
        # use .str to apply a string method(in this case I am using .replace) use [$,] to replace either of those with nothing "", and use
        # regex=True to use pandas built in regex integration
        df["Gross Earnings ($)"] = df["Gross Earnings ($)"].str.replace('[$,]', '', regex=True).astype(int)
        # we just want the movies that have grossed more than 100 million so I use a boolean mask 
        df = df[df["Gross Earnings ($)"] >= 100000000]
        # append it to the list
        dfList.append(df)
        
    # combine the list of dataframes    
    masterDF = pd.concat(dfList,ignore_index=True)
    # return the masterDF
    return(masterDF)

In [4]:
# set function output to the master DF
masterDF = getMovieData(["https://www.boxofficemojo.com/year/2024/?grossesOption=calendarGrosses&sort=gross","https://www.boxofficemojo.com/year/2023/?grossesOption=calendarGrosses&sort=gross","https://www.boxofficemojo.com/year/2022/?grossesOption=calendarGrosses&sort=gross","https://www.boxofficemojo.com/year/2021/?grossesOption=calendarGrosses&sort=gross","https://www.boxofficemojo.com/year/2020/?grossesOption=calendarGrosses&sort=gross","https://www.boxofficemojo.com/year/2019/?grossesOption=calendarGrosses&sort=gross","https://www.boxofficemojo.com/year/2018/?grossesOption=calendarGrosses&sort=gross","https://www.boxofficemojo.com/year/2017/?grossesOption=calendarGrosses&sort=gross","https://www.boxofficemojo.com/year/2016/?grossesOption=calendarGrosses&sort=gross","https://www.boxofficemojo.com/year/2015/?grossesOption=calendarGrosses&sort=gross"])

In [5]:
# create a movie list of the movie Title column using .tolist() so we can pass in the list of movies into some other functions 
movieList = masterDF["Movie Title"].tolist()

## OMDb API

### Get Movie Genre

In [8]:
# create function to get movie Genre, passing in the movieList and my API key
def getMovieGenreOMDb(movieList, apiKey):
    # store the baseURL
    baseURL = "http://www.omdbapi.com/"
    # create genre list
    genres = []

    # iterate through the movies
    for movie in movieList:
        # ChatGPT request("How to catch errors in api requests") wrap my request in a try except block to avoid errors instead of doing the
        # .status_code == 200 method
        try:
            # chatGP request("How to make my api calls more efficient") learned to use params which is a parameter in requests that allows you
            # pass in a dictionary where the key is a parameter in part of the url and the value is whatever variable comes next in the url
            # pass in the base url and then the params = the dictionary you created
            params = {
                "apiKey": apiKey,
                "t": movie
            }
            response = requests.get(baseURL, params=params).json()

            # Check to see if genre is in the API call
            if "Genre" in response:
                # append the genre to the genre list
                genres.append(response["Genre"])
                # if genre isnt found then append Genre not found to the list
            else:
                genres.append("Genre Not Found")

        # this exception is a python class that catches most runtime errors and prints the error out if there is one
        except Exception as error:
            # I am incorperating an f string into this where instead of doing string concatenation I am passing in the variables with {} 
            print(f"Error with {movie}: {str(error)}")

        # Sleep between requests to avoid hitting rate limits
        time.sleep(0.25)

    # return the genre
    return genres


In [9]:
# save the output of the function (which is a list) as a variable
movieGenres = getMovieGenreOMDb(movieList,"9007b3ae")

In [10]:
# make a new column in the dataframe by filling in with the genre list
masterDF["Movie Genre"] = movieGenres

In [11]:
# Animated vs Live Action
# use ternary conditional expression under a lambda function, a one liner "if else" statement where the value goes first, 
# then "if" condition, then the "else" value
# spliting on the the movie genre column to extract the Animation genre if there is one, if it doesn't have Animation we just print Live Action

masterDF["Animated/Live Action"] = masterDF["Movie Genre"].apply(
    lambda x: "Animation" if x.split(",")[0] == "Animation" else "Live Action")

In [12]:
# using the same lambda function setup we want to extract the primary genre
masterDF["Primary Movie Genre"] = masterDF["Movie Genre"].apply(
    lambda x: x.split(",")[1] if x.split(",")[0].strip() == "Animation" else x.split(",")[0])

### Get MPAA Movie Rating

In [14]:
# use same logic as the first OMDb api call but this time we are looking for MPAA ratings
def getMovieRatingOMDb(movieList, apiKey):
    baseURL = "http://www.omdbapi.com/"
    ratings = []
    
    for movie in movieList:
        try:
            params = {
                "apiKey": apiKey,
                "t": movie
            }
            response = requests.get(baseURL, params=params).json()

            if "Rated" in response:
                ratings.append(response["Rated"])

        except Exception as e:
            print(f"Error with {movie}: {str(e)}")

        
        time.sleep(0.25)

    return ratings

In [15]:
# set function output = to variable
Ratings = getMovieRatingOMDb(movieList,"9007b3ae")

In [16]:
# create new column with it
masterDF["Rated"] = Ratings

### Get IMDb Rating

In [18]:
# use same logic as the first and second OMDb api call but this time we are looking for imdb ratings
def getMovieimbdRatingOMDb(movieList, apiKey):
    baseURL = "http://www.omdbapi.com/"
    imbdRatings = []
    
    for movie in movieList:
        try:
            params = {
                "apiKey": apiKey,
                "t": movie
            }
            response = requests.get(baseURL, params=params).json()

           
            if "imdbRating" in response:
                imbdRatings.append(response["imdbRating"])

        except Exception as e:
            print(f"Error with {movie}: {str(e)}")

        time.sleep(0.25)

    return imbdRatings

In [19]:
# set function output = to variable
imdbRatings = getMovieimbdRatingOMDb(movieList,"9007b3ae")

In [20]:
# create new column with it
masterDF["imdb Rating"] = imdbRatings

## lastFM API

### Get Soundtrack Genre

In [23]:
def getGenre(movieList,apiKey):
    # intialize list
    soundtrackGenres = []
    # save rootURL
    rootURL = "http://ws.audioscrobbler.com/2.0/"
    # iterate through the movieList
    for movie in movieList:
        # create api call
        params = {
            "method": "album.search",
            "album": movie,
            "api_key": apiKey,
            "format": "json"
        }
        movieResponse = requests.get(rootURL, params=params).json()

        # artist = json response that gives us the first artist of the soundtrack we just searched for
        artist = movieResponse["results"]["albummatches"]["album"][0]["artist"]

        # create a list of genres to ignore (kind of like a stop words list)
        genresToIgnore = ["soundtrack","composer","composers","instrumental","video game music","score","seen live",
                          "composers","buffy the vampire slayer","ost","female vocalists","disney",
                          "movie score composers","lost","300","film composer","score","game"]
        
        # intialize a dictionary
        genreCounts = {}
        # create new api request to seach for the artist info
        paramsArtist = {
            "method":"artist.getinfo",
            "artist": artist,
            "api_key": apiKey,
            "format": "json"
        }
        artistInfo = requests.get(rootURL,params=paramsArtist).json()
        
        # if what we are looking for in in the json response...
        if "artist" in artistInfo and "tags" in artistInfo["artist"]:
            # we iterate through each tag (the genre, but lastfm calls them tags)
            for tag in artistInfo["artist"]["tags"]["tag"]:
                # genre = the tag name, use .lower just to avoid errors
                genre = tag["name"].lower()
                # if the genre isn't in the genres to ignore...
                if genre not in genresToIgnore:
                    # if the genre is in the genre dictiontionary we add one more value
                    if genre in genreCounts:
                        genreCounts[genre] += 1
                        # if it isn't in the dictionary we add it to it
                    else:
                        genreCounts[genre] = 1  

        # if the dictionary was succesfully made
        if genreCounts:
            # chatGPT request ("How to find the most common thing in a dictionary")
            # commonGenre = the most common genre in the dictionary (max looks for the maximum value in the dictionary)
            # pass in the dictionary and a key (the key tells you what the max method will compare to, in this case we use .get() method
            # to get the values of each of the keys in the dictionary and the max will find the maximum value)
            # this method can be used instead of intializing a counter and comparing each value manually in a for loop
            # I was originally using this because I was going to iterate through all the artists associated with the soundtrack, then find
            # the common genre, but that caused some api rate limiting issues so I abandoned that idea. I left this in just because I took
            # the time to learn how to do it.
            commonGenre = max(genreCounts, key=genreCounts.get)
            # append the most common genre of the artist to our list
            soundtrackGenres.append(commonGenre)
            # if nothing is found, we append genre not found
        else:
            soundtrackGenres.append("Genre not found")
            # use time.sleep to avoid rate limiting errors
        time.sleep(.25)
    # return the list
    return soundtrackGenres

In [24]:
# set function output = to variable
soundtracks = getGenre(movieList,"6d0beba266515ff9ce388828fa455f06")

In [25]:
# create new column with it
masterDF["Soundtrack Genre"] = soundtracks

### More Data Cleaning

In [27]:
# fix data set by manually entering the right imbd rating for row 228 (movie Insurgent)
masterDF.loc[228,"imdb Rating"] = 6.2

In [28]:
# manually enter rating for row 228 as well
masterDF.loc[228,"Rated"] = "PG-13"

In [29]:
# double check null columns
print(masterDF.isnull().sum())

Movie Title             0
Gross Earnings ($)      0
Release Date            0
Distributor             0
Year                    0
Movie Genre             0
Animated/Live Action    0
Primary Movie Genre     0
Rated                   0
imdb Rating             0
Soundtrack Genre        0
dtype: int64


In [30]:
# double check duplicated columns
print(masterDF.duplicated().sum())

0


In [31]:
# convert dataframe to csv file to save all the progress so far
masterDF.to_csv("Movie Dataset.csv", index=False)

In [ ]:
# re import it as a new dataframe so we can do some more data cleaning
df = pd.read_csv("data/Movie Dataset.csv")

### Condense Genres into a Primary Genre

In [34]:
df.head(1)

,Movie Title,Gross Earnings ($),Release Date,Distributor,Year,Movie Genre,Animated/Live Action,Primary Movie Genre,Rated,imdb Rating,Soundtrack Genre
0,Inside Out 2,652980194,Jun 14,Walt Disney Studios Motion Pictures,2024,"Animation, Adventure, Comedy",Animation,Adventure,PG,7.5,german


In [35]:
# create lists to condesnse all the niche genres into a main genre
allRapList = ["hip-hop","rap","rnb","trap","hip hop","emo rap"]
classical = ["classical","contemporary classical","orchestral","symphonic"]
nationalMusic = ["austria","australian","american","swedish","german","greek","japanese","brazilian"]
rock = ["indie rock","classic rock","post-rock","progressive rock","punk rock","rock","rock n roll","post-punk","pop punk","new wave","emo","grunge","doo wop"]
metal = ["metalcore","power metal","nu metal"]
pop = ["pop","bedroom pop"]
electronic = ["electronic","industrial","digicore","house","dubstep","lo-fi"]
jazz = ["jazz","soul"]

In [36]:
# condesnse all the niche genres into a main genre using .apply and a lambda function with ternary conditional if else logic
df["Primary Soundtrack Genre"] = df["Soundtrack Genre"].apply(
    lambda x: "rap" if x in allRapList else 
    "classical" if x in classical else 
    "national" if x in nationalMusic else 
    "rock" if x in rock else 
    "metal" if x in metal else
    "pop" if x in pop else
    "electronic" if x in electronic else
    "jazz" if x in jazz else x)

In [37]:
genreNotFound = df[df["Primary Soundtrack Genre"] == "Genre not found"]

In [38]:
genreNotFound

,Movie Title,Gross Earnings ($),Release Date,Distributor,Year,Movie Genre,Animated/Live Action,Primary Movie Genre,Rated,imdb Rating,Soundtrack Genre,Primary Soundtrack Genre
11,Kingdom of the Planet of the Apes,171130165,May 10,20th Century Studios,2024,"Action, Adventure, Drama",Live Action,Action,PG-13,6.9,Genre not found,Genre not found
23,The Super Mario Bros. Movie,574934330,Apr 5,Universal Pictures,2023,"Animation, Adventure, Comedy",Animation,Adventure,PG,7.0,Genre not found,Genre not found
29,Ant-Man and the Wasp: Quantumania,214504909,Feb 17,Walt Disney Studios Motion Pictures,2023,"Action, Adventure, Comedy",Live Action,Action,PG-13,6.0,Genre not found,Genre not found
30,John Wick: Chapter 4,187131806,Mar 24,Lionsgate,2023,"Action, Crime, Thriller",Live Action,Action,R,7.6,Genre not found,Genre not found
34,Mission: Impossible - Dead Reckoning Part One,172135383,Jul 12,Paramount Pictures,2023,"Action, Adventure, Thriller",Live Action,Action,PG-13,7.6,Genre not found,Genre not found
50,Jurassic World Dominion,376851080,Jun 10,Universal Pictures,2022,"Action, Adventure, Sci-Fi",Live Action,Action,PG-13,5.6,Genre not found,Genre not found
53,Thor: Love and Thunder,343256830,Jul 8,Walt Disney Studios Motion Pictures,2022,"Action, Adventure, Comedy",Live Action,Action,PG-13,6.2,Genre not found,Genre not found
54,Spider-Man: No Way Home,231808708,Dec 17,Sony Pictures Releasing,2022,"Action, Adventure, Fantasy",Live Action,Action,PG-13,8.2,Genre not found,Genre not found
56,Black Adam,168054237,Oct 21,Warner Bros.,2022,"Action, Adventure, Fantasy",Live Action,Action,PG-13,6.2,Genre not found,Genre not found
64,Spider-Man: No Way Home,572984769,Dec 17,Sony Pictures Releasing,2021,"Action, Adventure, Fantasy",Live Action,Action,PG-13,8.2,Genre not found,Genre not found


In [39]:
# I re ran this list of movies (from the genreNotFound) through my getsoundtrack function, but took out "instrumental" from the genres to ignore list
# all these movies came up with instrumental as the soundtrack 

In [40]:
# set instrumental as genre for Genre not found
df["Primary Soundtrack Genre"] = df["Primary Soundtrack Genre"].apply(lambda x: "instrumental" if x == "Genre not found" else x)

In [41]:
# create grouby to group the primary soundtrack genres and count each of them up
primarySoundtrackGenres = df.groupby("Primary Soundtrack Genre").agg({"Primary Soundtrack Genre": "count"})
# create new column name since i am aggregating by the same column we are grouping by
primarySoundtrackGenres.columns = ["Count"]
# sort values by count
primarySoundtrackGenres = primarySoundtrackGenres.sort_values("Count",ascending=False)
primarySoundtrackGenres
# just want to double check i did everything right

,Count
Primary Soundtrack Genre,
classical,45
electronic,44
instrumental,37
rap,22
pop,18
rock,15
ambient,15
national,8
metal,5


In [42]:
# save it finally as a csv
df.to_csv("Final Movie Dataset.csv", index=False)